In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

columns = ['title', 'director', 'writers', 'running_time', 'budget', 'box_office']

df = pd.DataFrame(columns=columns)
urls = ["https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring",
        'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Two_Towers',
        'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Return_of_the_King']

for u in urls:

    page = requests.get(u)

    soup = BeautifulSoup(page.content, "html.parser")

    units = {'million':1_000_000, 'billion':1_000_000_000}

    def convert(splitted_ls):
        return float(splitted_ls[0]) * units[splitted_ls[1][:-3]]

    df2 = {
        'title' : soup.find(id='firstHeading').text,
        'director' : soup.find('th', string='Directed by').findNext('td').findChild('a').text,
        'writers' : soup.find('th', string='Screenplay by').findNext('td').text.strip().split('\n'),
        'running_time' : int(soup.find('th', string='Running time').findNext('td').text.split()[0]),
        'budget' : convert(soup.find('th', string='Budget').findNext('td').text.replace('$','').split()),
        'box_office' : convert(soup.find('th', string='Box office').findNext('td').text.replace('$','').split()),
    }

    df.loc[len(df)] = df2.values()

df.to_json('lotr.json', orient='records')